In [1]:
from transitions import Machine

from transitions.extensions.asyncio import AsyncTimeout, AsyncMachine

import enum 
import logging
import asyncio

from pymongo import MongoClient
from bson.objectid import ObjectId
import uuid
from pprint import pprint

from datetime import datetime
from math import floor, ceil
import time
from time import sleep

import sys
import os

import zmq
import zmq.asyncio
# from s import Context
import json

logging.basicConfig(level=logging.INFO)

from dotenv import load_dotenv
load_dotenv()

import ccxt



In [3]:
exchanges = {
    'binance': getattr(ccxt, 'binance')({'apiKey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')}),
    'kucoin': getattr(ccxt, 'kucoin')({'apiKey': os.getenv('KUCOIN_API_KEY'), 'secret': os.getenv('KUCOIN_API_SECRET'), 'password': os.getenv('KUCOIN_API_PASSWORD')})
    # 'ftx': getattr(ccxt, 'ftx')({'apiKey': os.getenv('FTX_API_KEY'), 'secret': os.getenv('FTX_API_SECRET')})
}

In [4]:
exchange = exchanges['kucoin']
exchange.fetchOpenOrders(symbol='DOT/USDT')

[]

In [1]:
client = MongoClient()
db=client.fsmbot
# trades = {}



NameError: name 'MongoClient' is not defined

In [ ]:
# exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
exchange.fetchOpenOrders(symbol='DOTUSDT')

In [ ]:
# exchange.fetchOrderStatus('686722117', symbol='ALGOUSDT')
exchange.privateDeleteOrderList({'symbol': 'ADAUSDT', 'orderListId': '50105881'})
# exchange.fetchOrders(symbol='ALGOUSDT')

In [2]:
# test ticker zmq publisher
zContext = zmq.asyncio.Context()

socket = zContext.socket(zmq.SUB)
socket.connect("tcp://localhost:5556")
socket.setsockopt_string(zmq.SUBSCRIBE, 'TICKER')

while True:
    tickerString = await socket.recv_string()
    _, exchange, symbol, ticker = tickerString.split()
    ticker = json.loads(ticker)
    if symbol == 'ADAUSDT':
        print(ticker)


{'eventType': '24hrTicker', 'eventTime': 1636837455273, 'symbol': 'ADAUSDT', 'priceChange': '-0.01400000', 'priceChangePercent': '-0.679', 'weightedAvg': '2.05166421', 'prevDayClose': '2.06300000', 'curDayClose': '2.04900000', 'closeTradeQuantity': '1178.50000000', 'bestBid': '2.04800000', 'bestBidQnt': '34913.80000000', 'bestAsk': '2.04900000', 'bestAskQnt': '35645.10000000', 'open': '2.06300000', 'high': '2.08000000', 'low': '2.02100000', 'volume': '88564614.30000000', 'volumeQuote': '181704849.24120000', 'openTime': 1636751055237, 'closeTime': 1636837455237, 'firstTradeId': 316362322, 'lastTradeId': 316686750, 'totalTrades': 324429}
{'eventType': '24hrTicker', 'eventTime': 1636837456261, 'symbol': 'ADAUSDT', 'priceChange': '-0.01400000', 'priceChangePercent': '-0.679', 'weightedAvg': '2.05166414', 'prevDayClose': '2.06200000', 'curDayClose': '2.04900000', 'closeTradeQuantity': '1178.50000000', 'bestBid': '2.04800000', 'bestBidQnt': '38349.40000000', 'bestAsk': '2.04900000', 'bestAsk

CancelledError: 

In [ ]:
try:
    raise Exception('BOOOOM')
except BaseException as err:
    print(err)


BOOOOM


In [ ]:
# exchange.load_markets()

symbol = 'ADA/USDT'
market = exchange.market(symbol)
amount = 20.0

exchange.fetchTicker(symbol)
ticker = exchange.fetchTicker(symbol)
last_price = ticker['last']
last_price + 0.02

In [ ]:
# exchange.load_markets()
# exchange.market('ADAUSDT')
# exchange.amount_to_precision('ADAUSDT', 1.11111111111)
exchange.fetch_order('50102787', 'ADAUSDT')

In [15]:

symbol = 'ADA/USDT'
market = exchange.market(symbol)
amount = 20.0

exchange.fetchTicker(symbol)
ticker = exchange.fetchTicker(symbol)
last_price = ticker['last']

# price = 1.961
# stop_price = 1.921
# stop_limit_price = 1.901
price = last_price + 0.02
stop_price = last_price - 0.02
stop_limit_price = stop_price  - 0.02

response = exchange.private_post_order_oco({
    'symbol': market['id'],
    'side': 'SELL',  # SELL, BUY
    'quantity': exchange.amount_to_precision(symbol, amount),
    'price': exchange.price_to_precision(symbol, price),
    'stopPrice': exchange.price_to_precision(symbol, stop_price),
    'stopLimitPrice': exchange.price_to_precision(symbol, stop_limit_price),  # If provided, stopLimitTimeInForce is required
    'stopLimitTimeInForce': 'GTC',  # GTC, FOK, IOC
    # 'listClientOrderId': exchange.uuid(),  # A unique Id for the entire orderList
    # 'limitClientOrderId': exchange.uuid(),  # A unique Id for the limit order
    # 'limitIcebergQty': exchangea.amount_to_precision(symbol, limit_iceberg_quantity),
    # 'stopClientOrderId': exchange.uuid()  # A unique Id for the stop loss/stop loss limit leg
    # 'stopIcebergQty': exchange.amount_to_precision(symbol, stop_iceberg_quantity),
    # 'newOrderRespType': 'ACK',  # ACK, RESULT, FULL
})
print(response)

{'orderListId': '50091666', 'contingencyType': 'OCO', 'listStatusType': 'EXEC_STARTED', 'listOrderStatus': 'EXECUTING', 'listClientOrderId': 'sKVIkCZf3EYIHStRnMSszf', 'transactionTime': '1635801996544', 'symbol': 'ADAUSDT', 'orders': [{'symbol': 'ADAUSDT', 'orderId': '2551057287', 'clientOrderId': '72X4IpGOgPX01McQ5v5dA3'}, {'symbol': 'ADAUSDT', 'orderId': '2551057288', 'clientOrderId': 'AuAYiymOoI51VSlYDQmCcA'}], 'orderReports': [{'symbol': 'ADAUSDT', 'orderId': '2551057287', 'orderListId': '50091666', 'clientOrderId': '72X4IpGOgPX01McQ5v5dA3', 'transactTime': '1635801996544', 'price': '1.90000000', 'origQty': '20.00000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'STOP_LOSS_LIMIT', 'side': 'SELL', 'stopPrice': '1.92000000'}, {'symbol': 'ADAUSDT', 'orderId': '2551057288', 'orderListId': '50091666', 'clientOrderId': 'AuAYiymOoI51VSlYDQmCcA', 'transactTime': '1635801996544', 'price': '1.96000000', 'origQty': '20.00

In [19]:
exchange.cancel_order('2551057287', symbol)

{'info': {'orderListId': '50091666',
  'contingencyType': 'OCO',
  'listStatusType': 'ALL_DONE',
  'listOrderStatus': 'ALL_DONE',
  'listClientOrderId': 'sKVIkCZf3EYIHStRnMSszf',
  'transactionTime': '1635802090575',
  'symbol': 'ADAUSDT',
  'orders': [{'symbol': 'ADAUSDT',
    'orderId': '2551057287',
    'clientOrderId': '72X4IpGOgPX01McQ5v5dA3'},
   {'symbol': 'ADAUSDT',
    'orderId': '2551057288',
    'clientOrderId': 'AuAYiymOoI51VSlYDQmCcA'}],
  'orderReports': [{'symbol': 'ADAUSDT',
    'origClientOrderId': '72X4IpGOgPX01McQ5v5dA3',
    'orderId': '2551057287',
    'orderListId': '50091666',
    'clientOrderId': 'zxspYdsPnh7pSc94UpFxYa',
    'price': '1.90000000',
    'origQty': '20.00000000',
    'executedQty': '0.00000000',
    'cummulativeQuoteQty': '0.00000000',
    'status': 'CANCELED',
    'timeInForce': 'GTC',
    'type': 'STOP_LOSS_LIMIT',
    'side': 'SELL',
    'stopPrice': '1.92000000'},
   {'symbol': 'ADAUSDT',
    'origClientOrderId': 'AuAYiymOoI51VSlYDQmCcA',
    

In [3]:
1000.0 / 15 * 9.50

633.3333333333334